### 1. Install atlas CLI on the notebook environment.

In [ ]:
# CODE_BLOCK_1!wget https://fastdl.mongodb.org/mongocli/mongodb-atlas-cli_1.31.0_linux_x86_64.deb!dpkg -i mongodb-atlas-cli_1.31.0_linux_x86_64.deb

### 2. Authenticate the CLI against your Atlas organization.

In [ ]:
# CODE_BLOCK_2!atlas auth login

### 3. Create a new project called \"MySecureProj\"

In [ ]:
# CODE_BLOCK_3import getpassorg_id = getpass.getpass('Enter your org ID: ')!atlas projects create MySecureProj --orgId {org_id}

### 4. Create your challenge cluster

In [ ]:
# CODE_BLOCK_4project_id = getpass.getpass('Enter your project ID: ')!atlas clusters create MyNewCluster --provider AWS --region US_EAST_1 --tier M0 --projectId {project_id}

### 1. Add 'My current IP' temporary into the atlas project

In [ ]:
# CODE_BLOCK_5from datetime import datetime, timedeltadelete_after = (datetime.utcnow() + timedelta(hours=24)).isoformat() + 'Z'!atlas accessLists ...

### 2. Check that 'My current IP' was added:

In [ ]:
# CODE_BLOCK_6!atlas accessLists list --output json --projectId {project_id}

### 1. Let's create a SCRAM user: 'myUser' with User/Password authentication and assign it the 'readWriteAnyDatabase' role.

In [ ]:
# CODE_BLOCK_7!atlas dbusers create ...

### 2. Lets test our SCRAM user successful creation by performing the authentication process

In [ ]:
# CODE_BLOCK_8!pip install pymongo dnspython

In [ ]:
# CODE_BLOCK_9connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}username = 'myUser'password = 'mySecurePassword'new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')print(new_connection)from pymongo import MongoClientclient = MongoClient(new_connection)client.list_database_names()

### 3. Let's create  X509 user: 'myX509User' with User/Password authentication and assign it the 'readWriteAnyDatabase' role.

In [ ]:
# CODE_BLOCK_10!atlas dbusers create ...!atlas dbusers certs create ...  > /tmp/cert.pem

### 4. Let's test our X509 User

In [ ]:
# CODE_BLOCK_11username='myX509User'connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}new_connection = connection[1].replace('.net', '.net?authSource=%24external&authMechanism=MONGODB-X509')print(new_connection)from pymongo import MongoClientclient = MongoClient(new_connection,                     tlsCertificateKeyFile='/tmp/cert.pem')client.list_database_names()

### 1. Let's create a user: 'myAdmin' with User/Password authentication and assign it the 'readWriteAnyDatabase' role, but only for the 'MyNewCluster' database.

In [ ]:
# CODE_BLOCK_12!atlas dbusers create ...

### 2. Let's create a user: 'readOnlyUser' with read-only access to the 'salesDB' database.

In [ ]:
# CODE_BLOCK_13!atlas customDbRoles create ...!atlas dbusers create ...

### 3. Let's test that the 'readOnlyUser' cannot insert data into the 'salesDB' database.

In [ ]:
# CODE_BLOCK_14# Assuming you have the connection string and credentials for the readOnlyUserusername = 'readOnlyUser'password = 'readOnlyPass'connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')client = MongoClient(new_connection)db = client['salesDB']collection = db['mycollection']try:  data = {'name': 'John Doe', 'age': 30}  result = collection.insert_one(data)  print(f"Inserted document with ID: {result.inserted_id}")except Exception as e:  print(f"Error inserting data: {e}")

### 4. Let's test that the 'myAdmin' user can insert data into the 'salesDB' database.

In [ ]:
# CODE_BLOCK_15# Assuming you have the connection string and credentials for the myAdmin userusername = 'myAdmin'password = 'secureAdminPass'connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')client = MongoClient(new_connection)db = client['salesDB']collection = db['mycollection']try:  data = {'name': 'John Doe', 'age': 30}  result = collection.insert_one(data)  print(f"Inserted document with ID: {result.inserted_id}")except Exception as e:  print(f"Error inserting data: {e}")

### 1. Install the necessary packages.

In [ ]:
# CODE_BLOCK_16!pip install pymongo[encryption]!pip install pymongo!pip install requests

### 2. Set up the crypt_shared library.

In [ ]:
# CODE_BLOCK_17import osfrom pymongo import MongoClientfrom pymongo.encryption import Algorithm, ClientEncryption, QueryTypefrom pymongo.encryption_options import AutoEncryptionOptsfrom bson.codec_options import CodecOptionsfrom bson import json_utilimport jsonimport requestsimport platformimport tempfiledef setup_crypt_shared():    system = platform.system().lower()    if system == "linux":        url = "https://downloads.mongodb.com/linux/mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"        filename = "mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"    elif system == "darwin":        url = "https://downloads.mongodb.com/osx/mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"        filename = "mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"    elif system == "windows":        url = "https://downloads.mongodb.com/windows/mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"        filename = "mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"    else:        raise OSError("Unsupported operating system")    response = requests.get(url)    response.raise_for_status()    with tempfile.NamedTemporaryFile(delete=False, suffix=".tgz" if system != "windows" else ".zip") as tmp_file:        tmp_file.write(response.content)        tmp_file_path = tmp_file.name    extract_dir = tempfile.mkdtemp()    if system != "windows":        os.system(f"tar -xzf {tmp_file_path} -C {extract_dir}")        lib_path = os.path.join(extract_dir, "lib", "mongo_crypt_v1.so")    else:        os.system(f"powershell Expand-Archive -Path {tmp_file_path} -DestinationPath {extract_dir}")        lib_path = os.path.join(extract_dir, "bin", "mongo_crypt_v1.dll")    return lib_pathcrypt_shared_lib_path = setup_crypt_shared()print(f"Crypt shared library path: {crypt_shared_lib_path}")

### 3. Connect to your Atlas cluster.

In [ ]:
# CODE_BLOCK_18connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}username = 'myUser'password = 'mySecurePassword'new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')print(new_connection)from pymongo import MongoClientclient = MongoClient(new_connection)

### 4. Set up encryption key and providers, define the encrypted fields map, create an encrypted client, and set up the key vault.

In [ ]:
# CODE_BLOCK_19...

### 5. Insert an encrypted document.

In [ ]:
# CODE_BLOCK_20...

### 6. Query the encrypted collection.

In [ ]:
# CODE_BLOCK_21...

### 7. Query the collection without encryption.

In [ ]:
# CODE_BLOCK_22...